In [1]:
#!pip install map_boxes
#!pip install tqdm
#!pip install pycocotools

In [2]:
from map_boxes import mean_average_precision_for_boxes
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from pycocotools.coco import COCO

In [3]:
GT_JSON = 'detection/dataset/test.json'
PRED_CSV = './output_test.csv'

# load ground truth
with open(GT_JSON, 'r') as outfile:
    json_datas = (json.load(outfile))

# load prediction
pred_df = pd.read_csv(PRED_CSV)


In [35]:

key = 0
templist = [] #
for idx, item in enumerate(pred_df['PredictionString']):
    if type(item) == str :
        for v in np.array(item.strip().split(' ')).reshape(-1,6):
            tempdict = {}
            tempdict['image_id'] = idx
            tempdict['category_id'] = v[0]
            tempdict['area'] = str(float(v[4])*float(v[5]))
            tempdict['bbox'] = {0:v[2], 1:v[3], 2:v[4], 3:v[5]}
            tempdict['iscrowd'] = 0
            tempdict['id'] = key
            
            templist.append({key:tempdict})
            key=key+1


                    
#np.array(pred_df['PredictionString'][1].strip().split(' ')).reshape(-1,6)[1]
        
#cate         score   bb
#0         0.31056166 351.21313 228.14394 775.9386 706.2168 
#1         0.05101532 353.09308 215.32005 769.51447 719.37555 
#2         0.9990077614784241 347.39493 227.27557 782.5545 719.1181

{11: {'image_id': 0, 'category_id': '1', 'area': '243670.61668776898', 'bbox': {0: '324.40402', 1: '569.0784', 2: '393.71457', 3: '618.9017'}, 'iscrowd': 0, 'id': 11}}


In [55]:
def create_json(filename, option):
    result = {}

    result['info'] = json_datas['info']
    result['licenses'] = json_datas['licenses']
    result['images'] = json_datas['images']
    result['categories'] = json_datas['categories']
    
    templist = []
    key=0
    for idx, item in enumerate(pred_df['PredictionString']):
        if type(item) == str:
            for v in np.array(item.strip().split(' ')).reshape(-1,6):
                if int(v[0]) not in option:
                    continue
                tempdict = {}
                tempdict['image_id'] = idx
                tempdict['category_id'] = int(v[0])
                tempdict['area'] = float(v[4])*float(v[5])
                tempdict['bbox'] = [float(v[2]),float(v[3]), float(v[4]),float(v[5])]
                tempdict['iscrowd'] = 0
                tempdict['id'] = key

                templist.append(tempdict)
                key=key+1
    result['annotations'] = templist

    with open(f'./detection/dataset/{filename}.json', 'w') as outfile:
        json.dump(result, outfile)

In [59]:
create_json('pseudo1',[1])
create_json('pseudo2',[2])